In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
path=None
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        path=(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install transformers

In [3]:
import pandas as pd
import numpy as np


In [4]:
df=pd.read_csv(path,header=None)
text= df.iloc[:,0].values # Sentences
themes=df.iloc[:,1:]  # labels
print(text[0:2])
print(themes.head(2))

['Tires where delivered to the garage of my choice,the garage notified me when they had been delivered. A day and time was arranged with the garage and I went and had them fitted,a Hassel free experience.'
 'Easy Tyre Selection Process, Competitive Pricing and Excellent Fitting Service']
                        1                         2    3    4    5    6    7   \
0  garage service positive  ease of booking positive  NaN  NaN  NaN  NaN  NaN   
1  garage service positive  value for money positive  NaN  NaN  NaN  NaN  NaN   

    8    9    10   11   12   13   14  
0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
1  NaN  NaN  NaN  NaN  NaN  NaN  NaN  


In [5]:
themes_df = pd.DataFrame(index = range(0, themes.shape[0]), columns=['advisor/agent service', 'balancing', 'booking confusion', 'call wait time', 'change of date', 'change of time', 'damage', 'delivery punctuality', 'discount not applied', 'discounts', 'ease of booking', 'extra charges', 'facilities', 'failed payment', 'garage service', 'incorrect tyres sent', 'late notice', 'length of fitting', 'location', 'mobile fitter', "mobile fitter didn't arrive", 'no stock', 'refund', 'refund not actioned', 'refund timescale', 'response time', 'tyre age/dot code', 'tyre quality', 'value for money', 'wait time'])
# coverting positive label to 1 and negative label to 2
def encode_sentiment(s):
    if 'positive' in s:
        return (s.replace(' positive', ''), 1)
    else:
        return (s.replace(' negative', ''), 2)
        
for i, row in themes.iterrows():
  themes = row.dropna().tolist()
  
  final_theme = list(map(encode_sentiment, themes))
  for t in final_theme:
      themes_df.loc[i][t[0]] = t[1]  

themes_df=themes_df.fillna(0)
labels=themes_df.values
print(labels.shape)
label_name=themes_df.columns
print("Theme Categories:  ",label_name.values)

(10132, 30)
Theme Categories:   ['advisor/agent service' 'balancing' 'booking confusion' 'call wait time'
 'change of date' 'change of time' 'damage' 'delivery punctuality'
 'discount not applied' 'discounts' 'ease of booking' 'extra charges'
 'facilities' 'failed payment' 'garage service' 'incorrect tyres sent'
 'late notice' 'length of fitting' 'location' 'mobile fitter'
 "mobile fitter didn't arrive" 'no stock' 'refund' 'refund not actioned'
 'refund timescale' 'response time' 'tyre age/dot code' 'tyre quality'
 'value for money' 'wait time']


In [6]:
import torch
import torch.nn as nn
import random
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader
from transformers import BertTokenizer, BertModel
import torch.optim as optim
import torch.nn.functional as F

SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
seq_len=128

# tokenizing the sentenses using bert tokenizer it returns input_ids and attention_mask
def to_features(text):
  encoded = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=seq_len,
    padding='max_length',
    return_attention_mask=True,
    return_tensors='pt'
  )
  return encoded['input_ids'][:,:seq_len], encoded['attention_mask'][:,:seq_len]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
# Data Loader 
class MyDataset(Dataset):
  def __init__(self,data,labels):
    super(MyDataset,self).__init__()
    self.data=data
    self.labels=labels

  def __getitem__(self,index):
    input=self.data[index]
    input_ids,attn_mask=to_features(input)
    label=self.labels[index]
    return input_ids.squeeze(0),attn_mask.squeeze(0),label

  def __len__(self):
    return self.data.shape[0]

# splliting dataset into 70% and 30%
X_train, X_test, y_train, y_test =train_test_split(text, labels, test_size=0.30, random_state=42)
X_train, X_val, y_train, y_val =train_test_split(X_train, y_train, test_size=0.20, random_state=42)

train_data=MyDataset(X_train,y_train)
val_data=MyDataset(X_val,y_val)
test_data=MyDataset(X_test,y_test)

for input_ids,attn_mask,label in DataLoader(train_data,batch_size=2,shuffle=False):
  print("input_ids: ",input_ids)
  print("attention_mask:  ",attn_mask)
  print("label: ",label)
  break

input_ids:  tensor([[  101,  2204,  3643,  1012, 11414,  2832,  2499, 15299,  1012,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,    

In [9]:
#bert model laoding and freezing its weights
bert = BertModel.from_pretrained('bert-base-uncased')
for name, param in bert.named_parameters():                
  param.requires_grad = False

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [10]:
# Sentiment Calssifier Model
class ThemeAndSentiment(nn.Module):
  def __init__(self,seq_len,hidden_size):
    super().__init__()
    self.linear1=nn.Linear(hidden_size,512)
    self.linear2=nn.Linear(512,3)
  
  def forward(self,x):
    x=F.relu(self.linear1(x))
    x=self.linear2(x)
    return x

# Whole model with bert LSTM and Classifier
class Model(nn.Module):
  def __init__(self,bert,seq_len,hidden_size,output_size,dropout=.3):
      
    super().__init__()
    self.bert = bert
    self.output_size=output_size
    embedding_size = bert.config.to_dict()['hidden_size']
    self.lstm = nn.LSTM(embedding_size, hidden_size // 2,
                        num_layers=1, bidirectional=True, batch_first=True)
    self.sentiment=nn.ModuleList(ThemeAndSentiment(seq_len,hidden_size) for _ in range(output_size))
    self.dropout = nn.Dropout(dropout)
    self.criterion=nn.CrossEntropyLoss()
      
  def forward(self, input_ids,attn_mask):
    with torch.no_grad():
        embedded = self.bert(input_ids=input_ids,attention_mask=attn_mask)[0]

    lstm_out, _ = self.lstm(embedded)
    lstm_out=self.dropout(lstm_out)
    sentiment=[self.sentiment[i](lstm_out[:,-1,:]) for i in range(self.output_size) ]
    return sentiment

  def loss(self,prediction,target):
    loss=0
    for i in range(target.shape[1]):
      loss+=self.criterion(prediction[i],target[:,i])
    return loss


hidden_size=1024
output_size=30
dropout=.2
model=Model(bert,seq_len,hidden_size,output_size,dropout).to(device)

# for counting the number of trainable parameter
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')
#print(model)

The model has 21,041,242 trainable parameters


In [11]:
# coverting Logits to output by applying softmax and taking argmax
def to_softmax(preds):
    predicted=[]
    for pred in preds:
        pred=F.softmax(pred,dim=1)
        pred_max=torch.argmax(pred,dim=1).unsqueeze(1)
        predicted.append(pred_max)
    predicted=torch.cat(predicted,dim=1)
    return predicted


def accuracy(preds, target):
  predicted=to_softmax(preds)
  correct=(predicted.reshape(-1)==target.reshape(-1)).sum()
  acc=correct/predicted.reshape(-1).shape[0]
  return acc

from tqdm import tqdm

# Function to train the model on training dataset, returns loss and accuracy
def train(model,train_data, optimizer,batch_size):
    
  epoch_loss = 0
  epoch_acc = 0
  len=0
  model.train()
  
  for batch in tqdm(DataLoader(train_data,batch_size=batch_size)):
    optimizer.zero_grad()
    batch=tuple(t.to(device) for t in batch) 
    input_ids,attn_mask,target=batch

    predictions = model(input_ids,attn_mask)
    loss = model.loss(predictions,target)
    acc = accuracy(predictions, target)

    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()
    epoch_acc += acc.item()
    len+=1
      
  return epoch_loss / len, epoch_acc / len

# function to evaluate the model on validation dataset returns loss and accuracy
def eval(model,eval_data,batch_size):
    
  epoch_loss = 0
  epoch_acc = 0
  len=0
  model.eval()
  
  for batch in tqdm(DataLoader(eval_data,batch_size=batch_size)):
    with torch.no_grad():
      batch=tuple(t.to(device) for t in batch) 
      input_ids,attn_mask,target=batch

      predictions = model(input_ids,attn_mask)
      loss = model.loss(predictions,target)
      acc = accuracy(predictions, target)
      
      epoch_loss += loss.item()
      epoch_acc += acc.item()
      len+=1
      
  return epoch_loss / len, epoch_acc / len


# function to train and evaluate the model
def fit(epoch,lr=.001):
  optimizer = optim.Adam(model.parameters(),lr=.01)
  for epoch in range(epoch):
    train_loss,train_acc=train(model,train_data,optimizer,64)
    val_loss,val_acc=eval(model,val_data,128)
    print("epoch: %d  train_loss: %.3f  val_loss: %.3f  train_acc: %.3f  val_acc: %.3f"%(epoch,train_loss,val_loss,train_acc,val_acc))

In [12]:
fit(4,.01)

  0%|          | 0/89 [00:00<?, ?it/s]

epoch: 0  train_loss: 5.491  val_loss: 3.611  train_acc: 0.948  val_acc: 0.964


  0%|          | 0/89 [00:00<?, ?it/s]

epoch: 1  train_loss: 3.531  val_loss: 3.448  train_acc: 0.963  val_acc: 0.964


  0%|          | 0/89 [00:00<?, ?it/s]

epoch: 2  train_loss: 3.305  val_loss: 3.319  train_acc: 0.965  val_acc: 0.967


100%|██████████| 12/12 [00:06<00:00,  1.82it/s]

epoch: 3  train_loss: 3.185  val_loss: 3.176  train_acc: 0.966  val_acc: 0.967


In [13]:
_,test_acc=val_loss,val_acc=eval(model,test_data,256)
print("Test Accuracy:  %.3f " %test_acc)
    

100%|██████████| 12/12 [00:14<00:00,  1.18s/it]

Test Accuracy:  0.966 


In [14]:
# converts integer themes and sentiments back to corresponding Text label   
def convert_to_label(predicted):
    services={}
    for idx,i in enumerate(predicted):
        if i==1:
            services[label_name[idx]]="positive"
        elif i==2:
            services[label_name[idx]]="negative"
    return services

# Predict the themes and sentiments for given sentences
def predict(text):
    output=[]
    input_ids,attn_mask=[],[]
    for txt in text:
        ids,mask=to_features(txt)
        input_ids.append(ids)
        attn_mask.append(mask)
    input_ids=torch.cat(input_ids,dim=0)
    attn_mask=torch.cat(attn_mask,dim=0)
    predictions=model(input_ids.to(device),attn_mask.to(device))
    predicted=to_softmax(predictions).cpu().numpy()
    for pred in predicted:
        output.append(convert_to_label(pred))
    return output

txt,lbl=X_test[10:20].tolist(),y_test[10:20].tolist()

actual=[]
for i in lbl:
    actual.append(convert_to_label(i))
pred= predict(txt)
for i ,j in zip(actual,pred):
    print("actual: ",i,"   predicted: ",j)

actual:  {'location': 'positive', 'value for money': 'positive'}    predicted:  {'location': 'positive'}
actual:  {'value for money': 'positive'}    predicted:  {'value for money': 'positive'}
actual:  {'garage service': 'positive', 'location': 'positive', 'value for money': 'positive'}    predicted:  {'location': 'positive', 'value for money': 'positive'}
actual:  {'advisor/agent service': 'positive', 'ease of booking': 'positive', 'value for money': 'positive'}    predicted:  {'value for money': 'positive'}
actual:  {'delivery punctuality': 'negative'}    predicted:  {}
actual:  {}    predicted:  {}
actual:  {}    predicted:  {}
actual:  {}    predicted:  {}
actual:  {'location': 'positive', 'value for money': 'positive'}    predicted:  {'value for money': 'positive'}
actual:  {'ease of booking': 'positive', 'value for money': 'positive'}    predicted:  {'value for money': 'positive'}
